In [1]:
!pip install lightfm

In [2]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
import pandas as pd

In [3]:
ratings = pd.read_csv('ratings.csv') # Поставленные оценки
books = pd.read_csv('books.csv') # Информация о книгах
tags = pd.read_csv('tags.csv') # Информация о тегах
book_tags = pd.read_csv('book_tags.csv') # Книги с тегами

In [4]:
dict_map = dict(zip(books.goodreads_book_id,books.book_id))
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: dict_map[x])

book_tags[book_tags['goodreads_book_id']==5]

,goodreads_book_id,tag_id,count,id
300,5,11557,40087,18
301,5,11305,39330,18
302,5,8717,17944,18
303,5,33114,12856,18
304,5,30574,11909,18
...,...,...,...,...
395,5,20781,299,18
396,5,32345,298,18
397,5,12600,282,18
398,5,3379,277,18


In [5]:
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags.shape

(300738, 4)

In [6]:
!pip install scipy==1.10
from scipy.sparse import csr_matrix

In [7]:
ratings_matrix = csr_matrix(
    (ratings.rating,(ratings.user_id,ratings.book_id))
    ) # Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

In [8]:
meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id)))

In [9]:
ratings_matrix.mean()

0.007086188900997592

In [10]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

In [11]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

In [13]:
model = model.fit(
    train, # Обучающая выборка
    num_threads=6,
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True # Отображение обучения
)

Epoch: 100%|██████████| 10/10 [10:56<00:00, 65.68s/it]


In [14]:
prec_score = precision_at_k(
                     model,
                     test,
                     item_features = meta_matrix).mean()
print(prec_score)

0.02301363


In [ ]:
#scores = model.predict(<индекс интересующего пользователя>, np.arange(n_items),user_features=new_user_feature)  #Для предсказания рейтинга нового пользователя можно воспользоваться методом predict():